In [1]:
#Adding dependecies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
covid_data=pd.read_csv("data/country_vaccinations.csv")
covid_data.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/


In [3]:
population_vacc=covid_data[["country","people_fully_vaccinated_per_hundred"]].drop_duplicates(subset=['country'], keep="last")
population_vacc.head()

,country,people_fully_vaccinated_per_hundred
112,Afghanistan,0.46
274,Albania,12.68
403,Algeria,NaN
544,Andorra,21.25
656,Angola,1.56


In [39]:
selected=covid_data[["country","people_fully_vaccinated"]].groupby(["country",]).sum()
selected.head()

,people_fully_vaccinated
country,
Afghanistan,1203555.0
Albania,8446939.0
Algeria,0.0
Andorra,67309.0
Angola,2498465.0


In [38]:
country=covid_data["country"].unique()
iso_code=covid_data["iso_code"].unique()
#Creating new data frame
countries_summary=pd.DataFrame({"Country":country,
                                "ISO code": iso_code,
                              "%  of population vaccinated":population_vacc["people_fully_vaccinated_per_hundred"]})
countries_summary=countries_summary.set_index("Country")
countries_summary["Number of people fully vaccinated"]=selected["people_fully_vaccinated"]
countries_summary.head()

,ISO code,% of population vaccinated,Number of people fully vaccinated
Country,,,
Afghanistan,AFG,0.46,1203555.0
Albania,ALB,12.68,8446939.0
Algeria,DZA,NaN,0.0
Andorra,AND,21.25,67309.0
Angola,AGO,1.56,2498465.0
